In [1]:
import numpy as np
import xarray as xr

In [2]:
def generate_random_dates(size, start_date="2000-01-01", num_days=10000):
    base_date = np.datetime64(start_date)
    random_days = np.random.randint(0, num_days, size=size)
    return base_date + random_days.astype("timedelta64[D]")

In [4]:
s2_bands = np.random.rand(400_000, 55, 10).astype(np.float16)
s2_timestamps = generate_random_dates((400_000, 55))

np.savez_compressed("testing_xarray/s2_only.npz", s2_bands=s2_bands, s2_timestamps=s2_timestamps)

In [5]:
!du -h testing_xarray/s2_only.npz

412M	testing_xarray/s2_only.npz


In [6]:
# Carregar os dados
data = np.load("testing_xarray/s2_only.npz")
s2_bands = data["s2_bands"]  # shape: (400_000, 55, 10)
s2_timestamps = data["s2_timestamps"]     # shape: (400_000, 55)

n_samples, n_s2_timestamps, n_s2_bands = s2_bands.shape

# Criar o Dataset
ds = xr.Dataset(
    data_vars={
        "s2_reflectance": (("sample", "s2_time", "s2_band"), s2_bands),
        "s2_doy": (("sample", "s2_time"), s2_timestamps)
    },
    coords={
        "sample": np.arange(n_samples),
        "s2_time": np.arange(n_s2_timestamps),
        "s2_band": ["blue", "green", "red", "re1", "re2", "re3", "nir", "re4", "swir1", "swir2"]
    }
)

zarr_path = "testing_xarray/s2_only.zarr"

ds.to_zarr(zarr_path, mode="w", consolidated=True)

/home/mateus/git/AgriGEE.lite/.venv/lib/python3.11/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/mateus/git/AgriGEE.lite/.venv/lib/python3.11/site-packages/zarr/core/array.py:4282: UserWarning: The dtype `<U5` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  meta = AsyncArray._create_metadata_v3(
/home/mateus/git/AgriGEE.lite/.venv/lib/python3.11/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/mateus/git/AgriGEE.lite/.venv/lib/python3.11/site-packages/zarr/api/asynchro

In [7]:
!du -sh testing_xarray/s2_only.zarr

421M	testing_xarray/s2_only.zarr


In [8]:
ds = xr.open_zarr("testing_xarray/s2_only.zarr", consolidated=False)
ds

/home/mateus/git/AgriGEE.lite/.venv/lib/python3.11/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)


<xarray.Dataset> Size: 619MB
Dimensions:         (s2_band: 10, s2_time: 55, sample: 400000)
Coordinates:
  * s2_band         (s2_band) object 80B 'blue' 'green' ... 'swir1' 'swir2'
  * s2_time         (s2_time) int64 440B 0 1 2 3 4 5 6 ... 48 49 50 51 52 53 54
  * sample          (sample) int64 3MB 0 1 2 3 4 ... 399996 399997 399998 399999
Data variables:
    s2_doy          (sample, s2_time) datetime64[ns] 176MB ...
    s2_reflectance  (sample, s2_time, s2_band) float16 440MB ...

In [9]:
s2_bands = np.random.rand(400_000, 55, 10).astype(np.float16)
s2_timestamps = generate_random_dates((400_000, 55))

l8_bands = np.random.rand(400_000, 30, 6).astype(np.float16)
l8_timestamps = generate_random_dates((400_000, 30))

np.savez_compressed("testing_xarray/s2_l8_only.npz",
                    s2_bands=s2_bands, s2_timestamps=s2_timestamps,
                    l8_bands=l8_bands, l8_timestamps=l8_timestamps)

In [10]:
!du -h testing_xarray/s2_l8_only.npz

558M	testing_xarray/s2_l8_only.npz


In [11]:
# Carregar dados
data = np.load("testing_xarray/s2_l8_only.npz")
s2_bands = data["s2_bands"]  # (400_000, 55, 10)
s2_timestamps = data["s2_timestamps"]    # (400_000, 55)
l8_bands = data["l8_bands"]  # (400_000, 30, 6)
l8_timestamps = data["l8_timestamps"]    # (400_000, 30)

n_samples = s2_bands.shape[0]

# Criar Dataset
ds = xr.Dataset(
    data_vars={
        "s2_reflectance": (["sample", "s2_time", "s2_band"], s2_bands),
        "s2_doy": (["sample", "s2_time"], s2_timestamps),
        "l8_reflectance": (["sample", "l8_time", "l8_band"], l8_bands),
        "l8_doy": (["sample", "l8_time"], l8_timestamps),
    },
    coords={
        "sample": np.arange(n_samples),
        "s2_time": np.arange(s2_bands.shape[1]),
        "s2_band": ["blue", "green", "red", "re1", "re2", "re3", "nir", "re4", "swir1", "swir2"],
        "l8_time": np.arange(l8_bands.shape[1]),
        "l8_band": ["blue", "green", "red", "nir", "swir1", "swir2"]
    }
)

zarr_path = "testing_xarray/s2_l8_only.zarr"

ds.to_zarr(zarr_path, mode="w", consolidated=True)

/home/mateus/git/AgriGEE.lite/.venv/lib/python3.11/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/mateus/git/AgriGEE.lite/.venv/lib/python3.11/site-packages/zarr/core/array.py:4282: UserWarning: The dtype `<U5` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  meta = AsyncArray._create_metadata_v3(
/home/mateus/git/AgriGEE.lite/.venv/lib/python3.11/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/mateus/git/AgriGEE.lite/.venv/lib/python3.11/site-packages/zarr/core/array.p

In [13]:
ds

<xarray.Dataset> Size: 859MB
Dimensions:         (sample: 400000, s2_time: 55, s2_band: 10, l8_time: 30,
                     l8_band: 6)
Coordinates:
  * sample          (sample) int64 3MB 0 1 2 3 4 ... 399996 399997 399998 399999
  * s2_time         (s2_time) int64 440B 0 1 2 3 4 5 6 ... 48 49 50 51 52 53 54
  * s2_band         (s2_band) <U5 200B 'blue' 'green' 'red' ... 'swir1' 'swir2'
  * l8_time         (l8_time) int64 240B 0 1 2 3 4 5 6 ... 23 24 25 26 27 28 29
  * l8_band         (l8_band) <U5 120B 'blue' 'green' 'red' ... 'swir1' 'swir2'
Data variables:
    s2_reflectance  (sample, s2_time, s2_band) float16 440MB 0.8594 ... 0.07654
    s2_doy          (sample, s2_time) datetime64[s] 176MB 2006-05-12 ... 2012...
    l8_reflectance  (sample, l8_time, l8_band) float16 144MB 0.4963 ... 0.2832
    l8_doy          (sample, l8_time) datetime64[s] 96MB 2012-09-25 ... 2006-...

In [12]:
!du -sh testing_xarray/s2_l8_only.zarr

568M	testing_xarray/s2_l8_only.zarr
